In [1]:
from train_utils import *
import torch
from torch.utils.data import DataLoader
from dataset.dataset_loader import SNDataset, myNormalize, myToTensor, Augmentations
from torchvision import transforms
from sklearn.ensemble import RandomForestRegressor
import numpy as np
from sklearn.metrics import mean_squared_error

c:\Users\moi3n\miniconda3\envs\pytorchGPU\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os
os.getcwd()

NAFISEH = "Nafiseh"
MOIEN = "Moien"

if "d:" in os.getcwd():
    USER = MOIEN
elif "c:" in os.getcwd():
    USER = NAFISEH
else:
    raise Exception("Unknown user")

USER

'Moien'

In [3]:
# Setup device-agnostic code
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [4]:
#mynorm = myNormalize(img_bands_min_max =[[(0,7),(0,1)], [(7,12),(-1,1)], [(12), (-4,2963)], [(13), (0, 90)]], oc_min = 0, oc_max = 200)
mynorm = myNormalize(img_bands_min_max =[[(0,7),(0,1)], [(7,12),(-1,1)]], oc_min = 0, oc_max = 90)
my_to_tensor = myToTensor()
my_augmentation = Augmentations()
train_transform = transforms.Compose([mynorm, my_to_tensor])
test_transform = transforms.Compose([mynorm, my_to_tensor])

In [5]:
bands = [0,1,2,3,4,5,6,7,8,9,10,11]
if USER == MOIEN:
    train_ds = SNDataset('D:\python\SoilNet\dataset\l8_images\\train\\','D:\python\SoilNet\dataset\LUCAS_2015_all.csv',l8_bands=bands, transform=train_transform)
elif USER == NAFISEH:
    train_ds = SNDataset('C:\\Users\\nkakhani\\_Multimodal\\SoilNet-1\\dataset\\l8_images\\train',\
                        'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-1\\dataset\\LUCAS_2015_all.csv',l8_bands=bands, transform=train_transform) #Nafiseh 
if USER == MOIEN:
    test_ds = SNDataset('D:\python\SoilNet\dataset\l8_images\\test\\','D:\python\SoilNet\dataset\LUCAS_2015_all.csv',l8_bands=bands, transform=test_transform)
elif USER == NAFISEH:
    test_ds = SNDataset('C:\\Users\\nkakhani\\_Multimodal\\SoilNet-1\\dataset\\l8_images\\test',\
                        'C:\\Users\\nkakhani\\_Multimodal\\SoilNet-1\\dataset\\LUCAS_2015_all.csv',l8_bands=bands, transform=test_transform) #Nafiseh 

In [6]:
# CONFIG
NUM_WORKERS = 6 if USER == NAFISEH else 2
TRAIN_BATCH_SIZE = 32 if USER == NAFISEH else 4
TEST_BATCH_SIZE = 32 if USER == NAFISEH else 4
LEARNING_RATE = 1e-4 if USER == NAFISEH else 1e-4
NUM_EPOCHS = 100 if USER == NAFISEH else 10
LR_SCHEDULER = "step" # step, plateau or None
USE_SPATIAL_ATTENTION = True

In [7]:
train_dl = DataLoader(train_ds, batch_size=TRAIN_BATCH_SIZE, shuffle=True, num_workers=NUM_WORKERS)
test_dl = DataLoader(test_ds, batch_size=TEST_BATCH_SIZE, shuffle=False, num_workers=NUM_WORKERS)

In [8]:
# Preprocess the data using the DataLoader
X_processed = []
y_processed = []
for batch_idx, (features, target) in enumerate(train_dl):
    features_np = features.numpy()
    # Preprocess the features as needed
    features_processed = features_np.reshape(features_np.shape[0], -1)
    X_processed.append(features_processed)
    y_processed.append(target.numpy())

X_processed = np.concatenate(X_processed, axis=0)
y_processed = np.concatenate(y_processed, axis=0)

In [9]:
print(X_processed.shape, X_processed.dtype)
print(f"Memory size of the Train array is {X_processed.nbytes/(1024**2)} MB or {X_processed.nbytes/(1024**3)} GB" )
print(y_processed.shape)

(52, 49152) float32
Memory size of the Train array is 9.75 MB or 0.009521484375 GB
(52,)


In [10]:
# Train a RandomForestRegressor on the processed data
rfr = RandomForestRegressor(n_estimators=100, max_depth=5, random_state=42)
rfr.fit(X_processed, y_processed)

RandomForestRegressor(max_depth=5, random_state=42)

In [11]:
# Use the trained model to predict on a new image
new_image = test_ds[0][0].numpy()
new_image_processed = new_image.reshape(1, -1)
y_pred = rfr.predict(new_image_processed)
print("y_pred: ", y_pred[0], "|" ,"y_true: ", test_ds[0][1].numpy())

y_pred:  0.245731723090545 | y_true:  0.27333334


In [12]:
# Preprocess the data using the DataLoader
X_processed = []
y_processed = []
for batch_idx, (features, target) in enumerate(test_dl):
    features_np = features.numpy()
    # Preprocess the features as needed
    features_processed = features_np.reshape(features_np.shape[0], -1)
    X_processed.append(features_processed)
    y_processed.append(target.numpy())

X_processed = np.concatenate(X_processed, axis=0)
y_processed = np.concatenate(y_processed, axis=0)

In [14]:
y_pred = rfr.predict(X_processed)

In [16]:
rmse = np.sqrt(mean_squared_error(y_processed, y_pred))
print('RMSE:', rmse)

RMSE: 0.04451759898629307
